# Spark Structured Streaming with Stroom

### How to process JSON formatted messages containing event-logging events.

#### Prerequisites
This notebook is designed to work with a Stroom server process running on `localhost`, into which data from `EventGen` application has been ingested and indexed in the manner described in `stroom-analytic-demo`.

You must set the environmental variable `STROOM_API_KEY` to the API token associated with a suitably privileged Stroom user account before starting the Jupyter notebook server process.

In [1]:
from pyspark.sql.types import *
from pyspark.sql.functions import from_json, col, coalesce, unix_timestamp,lit,to_timestamp
from IPython.display import display
import time,os

#### Schema Discovery
It is necessary to specify the structure of the JSON data arriving on the topic.  This structure can be determined at runtime.

As the same format of data is also available via an indexed search using the `stroom-spark-datasource`, one way to determine the JSON schema is by interrogating the data held in the `Sample Index` Stroom index.

The specified pipeline is a Stroom Search Extraction Pipeline that uses the stroom:json XSLT function to create a JSON representation of the entire event.  This field is called "Json" by default but the name of the field that contains the JSON representation can (optionally) be changed with the parameter jsonField.

In this manner, all data is returned as a single JSON structure within the field **json**

In [2]:
spark = SparkSession \
    .builder \
    .appName("MyTestApp") \
    .getOrCreate()
schemaDf = spark.read.format('stroom.spark.datasource.StroomDataSource').load(
        token=os.environ['STROOM_API_KEY'],host='localhost',protocol='http',
        uri='api/stroom-index/v2',
        index='32dfd401-ee11-49b9-84c9-88c3d3f68dc2',pipeline='13143179-b494-4146-ac4b-9a6010cada89')
json_schema = spark.read.json(schemaDf.rdd.map(lambda row: row.json)).schema

json_schema

StructType(List(StructField(EventDetail,StructType(List(StructField(Authenticate,StructType(List(StructField(Action,StringType,true),StructField(Outcome,StructType(List(StructField(Permitted,StringType,true),StructField(Reason,StringType,true),StructField(Success,StringType,true))),true),StructField(User,StructType(List(StructField(Id,StringType,true))),true))),true),StructField(Process,StructType(List(StructField(Action,StringType,true),StructField(Command,StringType,true),StructField(Type,StringType,true))),true),StructField(TypeId,StringType,true))),true),StructField(EventId,StringType,true),StructField(EventSource,StructType(List(StructField(Client,StructType(List(StructField(HostName,StringType,true))),true),StructField(Device,StructType(List(StructField(HostName,StringType,true))),true),StructField(Generator,StringType,true),StructField(System,StructType(List(StructField(Environment,StringType,true),StructField(Name,StringType,true))),true),StructField(User,StructType(List(Struct

#### Working with JSON

Now it is possible to use the JSON structure to more usefully parse the events received from the topic.

Here a temporary view `userops` is created on the streaming dataframe, after which sql is used in order to run queries over the (infinite) view.

It is possible to run aggregations such as count/max with group by.  According to docs, only any aggregations that actually change should be output given `outputMode("update")`

`withWatermark` actually [only works for stateful processing](http://mail-archives.us.apache.org/mod_mbox/spark-user/201809.mbox/%3CK3n6KQSgUxM0tvsl6jW_ffj_9xuBQ3PoVE4npynWTAe6U-xarvBr2I1yqt2tVJ2OV6KEAwyrl5yWZXyWisDb-9RGi2TwzPxoxQPC5yCAd-I=@protonmail.com%3E), and therefore it has no effect in the code below. 

In [4]:
df = spark \
  .readStream \
  .format("kafka") \
  .option("kafka.bootstrap.servers", "localhost:9092") \
  .option("subscribe", "ANALYTIC-DEMO-UEBA") \
  .option("startingoffsets", "earliest")\
  .load()

wideDf = df.withColumn('json',col('value').cast('string')).\
    withColumn('evt', from_json(col('json'), json_schema)).\
    withColumn ('timestamp', to_timestamp(col('evt.EventTime.TimeCreated')).cast("timestamp")).\
    withColumn('operation', col('evt.EventDetail.TypeId')).\
    withColumn('streamid', col('evt.StreamId')).\
    withColumn('eventid', col('evt.EventId')).\
    withColumn ('user', coalesce(col('evt.EventSource.User.Id'),\
                                                col('evt.EventDetail.Authenticate.User.Id'))).\
    withWatermark("timestamp", "30 minutes").\
    dropDuplicates(["eventid", "streamid"])


wideDf.printSchema()

wideDf.createOrReplaceTempView("userops")

#userCounts=spark.sql("select user from userops")

userCounts = spark.sql("select user,operation, count (streamid, eventid) as events from userops \
                    group by user,operation")

#N.B. You cannot order results of the above, unless using outputMode("complete")
               
query = userCounts \
 .writeStream \
    .outputMode("update") \
    .format("console") \
    .start()

query.awaitTermination()

root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)
 |-- json: string (nullable = true)
 |-- evt: struct (nullable = true)
 |    |-- EventDetail: struct (nullable = true)
 |    |    |-- Authenticate: struct (nullable = true)
 |    |    |    |-- Action: string (nullable = true)
 |    |    |    |-- Outcome: struct (nullable = true)
 |    |    |    |    |-- Permitted: string (nullable = true)
 |    |    |    |    |-- Reason: string (nullable = true)
 |    |    |    |    |-- Success: string (nullable = true)
 |    |    |    |-- User: struct (nullable = true)
 |    |    |    |    |-- Id: string (nullable = true)
 |    |    |-- Process: struct (nullable = true)
 |    |    |    |-- Action: string (nullable = true)
 |    |    |    |-- Command: string (nullable = true)

KeyboardInterrupt: 